In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import geopandas as gpd

plt.style.use('ggplot')

<table border=1 border-collapse="collapse" width="100%">
  <thead>
    <th>
      <img src="https://static.wikia.nocookie.net/logopedia/images/5/52/EPM2007.png/revision/latest?cb=20210505181105&path-prefix=es" width="320px" height="270px" />
    </th>
    <th>
      <h1>Análisis de Datos para la Gestión de Residuos de Construcción y Demolición (RCD) en Antioquia</h1>
    </th>
  </thead>
</table>

In [ ]:
data = pd.read_csv('./DATA/itemsEjec.csv')

data.head()

In [ ]:
subDict = {
    'Municipio': list(),
    'Subregion': list(),
    'Aprovechamiento': list(),
    'DispFinal': list(),
    'Percent': list()
}

for i in range(0, len(data), 1):
    row = data.iloc[i]
    subDict['Municipio'].append(row['Municipio'])
    subDict['Subregion'].append(row['Subregion'])
    more = 0
    less = 0
    # TIERRAS
    if row['areaTierras']:
        more += row['tierras']
    else:
        less += row['tierras']
    # ROCA
    if row['areaRoca']:
        more += row['roca']
    else:
        less += row['roca']
    # CONCRETO
    if row['areaConcreto']:
        more += row['concreto']
    else:
        less += row['concreto']
    # PAVIMENTO
    if row['areaPavimento']:
        more += row['pavimento']
    else:
        less += row['pavimento']
    subDict['Aprovechamiento'].append(more)
    subDict['DispFinal'].append(less)
    subDict['Percent'].append(100*more/(more + less))

df = pd.DataFrame(subDict)

sns.barplot(
    data=df[df['Percent'] > 0].sort_values(by=['Subregion', 'Municipio'], ascending=[False, True]),
    x='Percent',
    y='Municipio',
    hue='Subregion',
    errorbar=None,
    palette=sns.color_palette(['#154D71', '#E43636', '#386641'])
)

plt.title('Porcentaje de aprovechamiento por municipio')
plt.xlabel('% de aprovechamiento')

plt.show()

In [ ]:
percent = 100*df['Aprovechamiento'].sum() / (df['Aprovechamiento'].sum() + df['DispFinal'].sum())

print('El porcentaje general de aprovechamiento de RCD en Antioquia fue del ' + str(round(percent, 2)) + '%')

In [ ]:
gdf = gpd.read_file('./QGIS/Antioquia.gpkg', layer='Municipios')

noEPM = set(gdf['MpNombre']) - set(df['Municipio'])

for i in list(noEPM):
    subDict['Municipio'].append(i)
    subDict['Subregion'].append(list(gdf[gdf['MpNombre'] == 'MEDELLIN']['Mp_SUBREGI'])[0])
    subDict['Aprovechamiento'].append(0)
    subDict['DispFinal'].append(0)
    subDict['Percent'].append(0)

df = pd.DataFrame(subDict).groupby(['Municipio', 'Subregion']).mean()

gdf = gdf.merge(df, left_on='MpNombre', right_on='Municipio', sort=True)

gdf.describe()

In [ ]:
gdf.plot(
    column='Percent',
    cmap='Greens',
    legend=True,
    vmin=0, vmax=100
)

plt.title('Distribución espacia del aprovechamiento de los RCD')

plt.show()